In [117]:
import nibabel as nib
import numpy as np
surf = nib.load('/localscratch/hippunfold/hippunfold/resources/unfold_template_hipp/tpl-avg_space-unfold_den-0p5mm_midthickness.surf.gii')


In [2]:
surf

In [6]:
surf.darrays[0].print_summary()

Intent:  NIFTI_INTENT_TRIANGLE
DataType:  NIFTI_TYPE_INT32
ArrayIndexingOrder:  ColumnMajorOrder
Dimensionality:  2
Dimensions:  [14266, 3]
Encoding:  GZipBase64Binary
Endian:  LittleEndian
ExternalFileName:  
ExternalFileOffset:  0
----
Coordinate System:
Dataspace:  NIFTI_XFORM_UNKNOWN
XFormSpace:  NIFTI_XFORM_UNKNOWN
Affine Transformation Matrix: 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
None


In [64]:
surf.darrays[1].print_summary()

Intent:  NIFTI_INTENT_POINTSET
DataType:  NIFTI_TYPE_FLOAT32
ArrayIndexingOrder:  ColumnMajorOrder
Dimensionality:  2
Dimensions:  [7262, 3]
Encoding:  GZipBase64Binary
Endian:  LittleEndian
ExternalFileName:  
ExternalFileOffset:  0
----
Coordinate System:
Dataspace:  NIFTI_XFORM_UNKNOWN
XFormSpace:  NIFTI_XFORM_UNKNOWN
Affine Transformation Matrix: 
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
None


In [95]:
triangles = surf.darrays[0].data
triangles

array([[ 242,  145,  240],
       [ 305,  242,  240],
       [ 358,    4,   29],
       ...,
       [4491, 4490, 4474],
       [4473, 4490, 4472],
       [4490, 4489, 4472]], dtype=int32)

In [4]:
striatum_file = 'diffparc/resources/tpl-MNI152NLin2009cAsym/CIT168_prob/striatum_prob.nii.gz'

striatum_nib = nib.load(striatum_file)


In [118]:
import pyvista as pv
from vtk import vtkNIFTIImageReader


class NIFTIReader(pv.BaseReader):
    _class_reader = vtkNIFTIImageReader


reader = NIFTIReader(striatum_file)
vol = reader.read()
print(vol)
surface = vol.contour([0.5])
print(surface)
#surface.save(snakemake.output.vtk, binary=False)


UniformGrid (0x7f2662be0640)
  N Cells:	8404992
  N Points:	8530021
  X Bounds:	0.000e+00, 1.920e+02
  Y Bounds:	0.000e+00, 2.280e+02
  Z Bounds:	0.000e+00, 1.920e+02
  Dimensions:	193, 229, 193
  Spacing:	1.000e+00, 1.000e+00, 1.000e+00
  N Arrays:	1

PolyData (0x7f2662be05e0)
  N Cells:	32448
  N Points:	16236
  X Bounds:	5.972e+01, 1.314e+02
  Y Bounds:	9.375e+01, 1.580e+02
  Z Bounds:	6.234e+01, 1.050e+02
  N Arrays:	1



In [119]:
new_points = surface.points
np.array(new_points)

array([[ 81.      , 138.      ,  62.9661  ],
       [ 82.      , 138.      ,  62.712887],
       [ 83.      , 138.      ,  62.806396],
       ...,
       [114.255646, 130.      , 104.      ],
       [114.      , 130.72427 , 104.      ],
       [114.      , 130.      , 104.23731 ]], dtype=float32)

In [120]:
new_faces = surface.faces
new_faces = new_faces.reshape((int(new_faces.shape[0]/4),4))
new_faces.shape


(32448, 4)

In [121]:
new_faces = new_faces[:,1:4]
new_faces

array([[    0,    26,    33],
       [    0,     1,    27],
       [   26,     0,    27],
       ...,
       [16229, 16230, 16231],
       [16232, 16235, 16234],
       [16233, 16234, 16235]])

In [122]:
new_gii = nib.GiftiImage()

In [123]:
new_tri_darray = nib.gifti.GiftiDataArray(data=new_faces, 
                         intent='NIFTI_INTENT_TRIANGLE',
                         datatype='NIFTI_TYPE_INT32')
                         

In [124]:
new_tri_darray.print_summary()

Intent:  NIFTI_INTENT_TRIANGLE
DataType:  NIFTI_TYPE_INT32
ArrayIndexingOrder:  RowMajorOrder
Dimensionality:  2
Dimensions:  [32448, 3]
Encoding:  GZipBase64Binary
Endian:  LittleEndian
ExternalFileName:  
ExternalFileOffset:  0
----
Coordinate System:
Dataspace:  NIFTI_XFORM_UNKNOWN
XFormSpace:  NIFTI_XFORM_UNKNOWN
Affine Transformation Matrix: 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
None


In [125]:
new_points_darray = nib.gifti.GiftiDataArray(data=new_points,
                         intent='NIFTI_INTENT_POINTSET',
                         datatype='NIFTI_TYPE_FLOAT32')

In [127]:
new_points_darray.print_summary()

Intent:  NIFTI_INTENT_POINTSET
DataType:  NIFTI_TYPE_FLOAT32
ArrayIndexingOrder:  RowMajorOrder
Dimensionality:  2
Dimensions:  [16236, 3]
Encoding:  GZipBase64Binary
Endian:  LittleEndian
ExternalFileName:  
ExternalFileOffset:  0
----
Coordinate System:
Dataspace:  NIFTI_XFORM_UNKNOWN
XFormSpace:  NIFTI_XFORM_UNKNOWN
Affine Transformation Matrix: 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
None


In [128]:
new_gifti = nib.GiftiImage()
new_gifti.add_gifti_data_array(new_points_darray)
new_gifti.add_gifti_data_array(new_tri_darray)

In [129]:
new_gifti.to_filename('test_striatum.surf.gii')